<a href="https://colab.research.google.com/github/thibi11/Brain-tumor-classification/blob/main/Brain_tumor_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import gradio as gr
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
# 📁 STEP 2: Mount Google Drive (if dataset is there)
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/Brain Tumor Segmentation/Training'

CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']
data = []
labels = []

# ✅ Load and resize images
for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading {category}...")
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(category)
        except Exception as e:
            pass

# Preprocessing
data = np.array(data) / 255.0
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# Function to make predictions
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    label = lb.inverse_transform(prediction)
    return label[0]

# Create Gradio interface
interface = gr.Interface(fn=predict_image,
                         inputs=gr.Image(type='pil'),
                         outputs=gr.Label(num_top_classes=4),
                         title="Brain Tumor Classifier",
                         description="Upload an image to classify a brain tumor as glioma, meningioma, notumor, or pituitary.")
interface.launch()


Loading glioma...
Loading meningioma...
Loading notumor...
Loading pituitary...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 209s 717ms/step - accuracy: 0.6351 - loss: 0.9031 - val_accuracy: 0.2625 - val_loss: 6.3698
Epoch 2/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 261s 713ms/step - accuracy: 0.7452 - loss: 0.6524 - val_accuracy: 0.4296 - val_loss: 1.6196
Epoch 3/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 264s 721ms/step - accuracy: 0.8161 - loss: 0.5137 - val_accuracy: 0.7087 - val_loss: 0.6876
Epoch 4/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 267s 739ms/step - accuracy: 0.8257 - loss: 0.4621 - val_accuracy: 0.7900 - val_loss: 0.5946
Epoch 5/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 260s 734ms/step - accuracy: 0.8507 - loss: 0.4038 - val_accuracy: 0.8565 - val_loss: 0.4266
Epoch 6/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 209s 730ms/step - accuracy: 0.8665 - loss: 0.3711 - val_accuracy: 0.8399 - val_loss: 0.4023
Epoch 7/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 207s 725ms/step - accuracy: 0.8701 - loss: 0.3460 - val_accuracy: 0.7113 - val_loss: 1.1864
Epoch 8/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 261s 720ms/step - accuracy: 0.8857 -